In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split


# 🔥 Check if GPU is available


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


## 📌 1️⃣ Define Transformations (Resizing + Normalization)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize (0-1 → -1 to 1)
])

## 📌 2️⃣ Load Dataset from Directory


In [ ]:
dataset = datasets.ImageFolder(root="./WondersofWorld", transform=transform)

In [63]:
img,label = dataset[3800]
img.shape

torch.Size([3, 224, 224])

In [64]:
print(dataset.class_to_idx)
len(dataset)

{'burj_khalifa': 0, 'chichen_itza': 1, 'christ_the_reedemer': 2, 'eiffel_tower': 3, 'great_wall_of_china': 4, 'machu_pichu': 5, 'pyramids_of_giza': 6, 'roman_colosseum': 7, 'statue_of_liberty': 8, 'stonehenge': 9, 'taj_mahal': 10, 'venezuela_angel_falls': 11}


3846

In [18]:
# 🔹 80% Training, 20% Validation Split
train_size = int(0.8 * len(dataset))  # 80% data for training
val_size = len(dataset) - train_size  # 20% data for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [19]:
train_dataset

## 📌 3️⃣ Create DataLoaders (With Shuffle)


In [ ]:
batch_size = 32  # Same as before


In [11]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {train_size}")
print(f"Validation samples: {val_size}")


Training samples: 3076
Validation samples: 770


## 📌 4️⃣ Define CNN Model


In [ ]:

class CNN(nn.Module):
    def __init__(self, num_classes=12):  # 12 classes
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # Adjusted for 224x224 input
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

        self.dropout = nn.Dropout(0.5)  # Prevent overfitting

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## 📌 5️⃣ Initialize Model and Move to GPU


In [ ]:
model = CNN(num_classes=12).to(device)


## 📌 6️⃣ Define Loss & Optimizer


In [ ]:
criterion = nn.CrossEntropyLoss()  # Multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam Optimizer


## 📌 7️⃣ Training Loop


In [ ]:
epochs = 2  # Change as needed

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")


Epoch 1/2, Loss: 2.1270, Train Accuracy: 0.2477
Epoch 2/2, Loss: 1.7087, Train Accuracy: 0.4119


## 📌 8️⃣ Validation Loop


In [ ]:

model.eval()  # Set model to evaluation mode
correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():  # No gradients for validation
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_accuracy = correct / total
print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}")

Validation Loss: 1.5811, Validation Accuracy: 0.4818
